# 👥 Cohort & Segmentation Analysis

**UIDAI Data Hackathon 2026 - Behavioral Deep-Dive**

---

## 📋 Executive Overview

This notebook explores **behavioral patterns** across different user cohorts and segments:

### 🎯 Analysis Focus:
1. **Children vs Adults** - Update behavior comparison
2. **Transition Matrices** - State-to-state movement patterns
3. **Biometric-Demographic Correlation** - Cross-dataset analysis
4. **Update Journey Flow** - Temporal progression analysis
5. **Multi-dimensional Segmentation** - Age, geography, time, update type

---

**Objective**: Uncover hidden patterns in user behavior to inform targeted interventions and service improvements.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Add src to path
import sys
sys.path.append('../src')

# Import custom modules
from cohort_analysis import CohortAnalyzer, SegmentationEngine
from advanced_visualizations import AdvancedVisualizer

print("✓ All libraries loaded")
print("✓ Custom modules: cohort_analysis, advanced_visualizations")

In [ ]:
# Load datasets
enrolment_df = pd.read_parquet('../outputs/enrolment_processed.parquet')
demographic_df = pd.read_parquet('../outputs/demographic_processed.parquet')
biometric_df = pd.read_parquet('../outputs/biometric_processed.parquet')

print(f"✓ Enrolment: {len(enrolment_df):,} records")
print(f"✓ Demographic: {len(demographic_df):,} records")
print(f"✓ Biometric: {len(biometric_df):,} records")

---

## 👶👨 Section 1: Children vs Adults Analysis

### Behavioral Pattern Comparison

In [ ]:
# Initialize Cohort Analyzer
cohort_analyzer = CohortAnalyzer()

# Compare children vs adults
comparison = cohort_analyzer.compare_update_patterns(demographic_df)

print("=" * 80)
print(" " * 25 + "👶👨 CHILDREN VS ADULTS COMPARISON")
print("=" * 80)

for segment_name in ['children', 'adults']:
    if segment_name not in comparison:
        continue
    
    segment_data = comparison[segment_name]
    
    print(f"\n📊 {segment_name.upper()} SEGMENT:")
    print("-" * 80)
    print(f"  Count:                    {segment_data['count']:>25,}")
    print(f"  Percentage:               {segment_data['percentage']:>24.1f}%")
    print(f"  Average Age:              {segment_data['avg_age']:>25.1f} years")
    print(f"  Median Age:               {segment_data['median_age']:>25.1f} years")
    print(f"  States Covered:           {segment_data['states_covered']:>25,}")
    print(f"  Districts Covered:        {segment_data['districts_covered']:>25,}")
    
    if segment_data['update_type_distribution']:
        print(f"\n  Update Type Distribution:")
        for update_type, pct in list(segment_data['update_type_distribution'].items())[:5]:
            print(f"    {update_type:.<45} {pct:>12.1%}")

if 'differences' in comparison:
    print(f"\n📈 KEY DIFFERENCES:")
    print("-" * 80)
    diff = comparison['differences']
    print(f"  Adult to Child Ratio:     {diff['count_ratio']:>25.2f}x")
    print(f"  Age Difference:           {diff['avg_age_diff']:>25.1f} years")
    print(f"  State Coverage Diff:      {diff['geographic_coverage_diff']['states']:>25}")
    print(f"  District Coverage Diff:   {diff['geographic_coverage_diff']['districts']:>25}")

print("=" * 80)

In [ ]:
# Visualize comparison between children and adults
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Population Distribution', 'Average Age', 'Geographic Coverage', 'Top Update Types'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Population distribution
segments = ['Children', 'Adults']
counts = [comparison['children']['count'], comparison['adults']['count']]
percentages = [comparison['children']['percentage'], comparison['adults']['percentage']]

fig.add_trace(
    go.Bar(x=segments, y=counts, text=[f"{p:.1f}%" for p in percentages], 
           textposition='auto', name='Count', marker_color=['#FF6B6B', '#4ECDC4']),
    row=1, col=1
)

# Average age
ages = [comparison['children']['avg_age'], comparison['adults']['avg_age']]
fig.add_trace(
    go.Bar(x=segments, y=ages, text=[f"{a:.1f}" for a in ages], 
           textposition='auto', name='Avg Age', marker_color=['#95E1D3', '#F38181']),
    row=1, col=2
)

# Geographic coverage
states_cov = [comparison['children']['states_covered'], comparison['adults']['states_covered']]
districts_cov = [comparison['children']['districts_covered'], comparison['adults']['districts_covered']]

fig.add_trace(
    go.Bar(x=segments, y=states_cov, name='States', marker_color='#AA96DA'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=segments, y=districts_cov, name='Districts', marker_color='#FCBAD3'),
    row=2, col=1
)

# Top update types
if comparison['children']['update_type_distribution'] and comparison['adults']['update_type_distribution']:
    children_types = list(comparison['children']['update_type_distribution'].items())[:3]
    adults_types = list(comparison['adults']['update_type_distribution'].items())[:3]
    
    update_types = list(set([t[0] for t in children_types + adults_types]))
    
    children_vals = [comparison['children']['update_type_distribution'].get(t, 0) * 100 for t in update_types]
    adults_vals = [comparison['adults']['update_type_distribution'].get(t, 0) * 100 for t in update_types]
    
    fig.add_trace(
        go.Bar(x=update_types, y=children_vals, name='Children %', marker_color='#FFD93D'),
        row=2, col=2
    )
    fig.add_trace(
        go.Bar(x=update_types, y=adults_vals, name='Adults %', marker_color='#6BCB77'),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="👶👨 Children vs Adults: Comprehensive Comparison", showlegend=True)
fig.update_xaxes(title_text="Segment", row=1, col=1)
fig.update_xaxes(title_text="Segment", row=1, col=2)
fig.update_xaxes(title_text="Segment", row=2, col=1)
fig.update_xaxes(title_text="Update Type", row=2, col=2)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Age (years)", row=1, col=2)
fig.update_yaxes(title_text="Coverage Count", row=2, col=1)
fig.update_yaxes(title_text="Percentage", row=2, col=2)

fig.show()

---

## Section 2: State Transition Matrices 🔄

**Understanding Update Lifecycle Transitions**

Analyzing how users move through different update states over time

In [ ]:
# Create transition matrix for enrolment data
transition_data = cohort_analyzer.create_transition_matrix(enrolment_df)

print("=" * 80)
print(" " * 25 + "🔄 STATE TRANSITION MATRIX")
print("=" * 80)

if 'transition_matrix' in transition_data:
    tm = transition_data['transition_matrix']
    print("\nTransition Matrix (rows = from state, columns = to state):")
    print(tm.to_string())
    
    print(f"\n📊 TRANSITION STATISTICS:")
    print("-" * 80)
    print(f"  Total Transitions:        {transition_data['total_transitions']:>25,}")
    print(f"  Unique Users:             {transition_data['unique_users']:>25,}")
    print(f"  Avg Transitions/User:     {transition_data['avg_transitions_per_user']:>25.2f}")
    
    if 'state_counts' in transition_data:
        print(f"\n  State Distribution:")
        for state, count in sorted(transition_data['state_counts'].items(), key=lambda x: x[1], reverse=True):
            pct = (count / transition_data['unique_users']) * 100
            print(f"    {state:.<45} {count:>10,} ({pct:>5.1f}%)")

print("=" * 80)

In [ ]:
# Visualize transition matrix as heatmap
if 'transition_matrix' in transition_data:
    tm = transition_data['transition_matrix']
    
    fig = go.Figure(data=go.Heatmap(
        z=tm.values,
        x=tm.columns,
        y=tm.index,
        colorscale='YlOrRd',
        text=tm.values,
        texttemplate='%{text:.3f}',
        textfont={"size": 10},
        colorbar=dict(title="Probability")
    ))
    
    fig.update_layout(
        title='🔄 State Transition Probability Matrix',
        xaxis_title='To State',
        yaxis_title='From State',
        height=600,
        width=800
    )
    
    fig.show()
    
    # Create Sankey diagram for top transitions
    # Extract top 15 transitions
    transitions = []
    for from_state in tm.index:
        for to_state in tm.columns:
            if tm.loc[from_state, to_state] > 0.01:  # Only show transitions > 1%
                transitions.append({
                    'from': from_state,
                    'to': to_state,
                    'value': tm.loc[from_state, to_state]
                })
    
    transitions_sorted = sorted(transitions, key=lambda x: x['value'], reverse=True)[:15]
    
    if transitions_sorted:
        # Create unique node list
        nodes = list(set([t['from'] for t in transitions_sorted] + [t['to'] for t in transitions_sorted]))
        node_dict = {node: idx for idx, node in enumerate(nodes)}
        
        fig_sankey = go.Figure(data=[go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=nodes,
                color=['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', 
                       '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd'][:len(nodes)]
            ),
            link=dict(
                source=[node_dict[t['from']] for t in transitions_sorted],
                target=[node_dict[t['to']] for t in transitions_sorted],
                value=[t['value'] * 100 for t in transitions_sorted],
                label=[f"{t['value']:.1%}" for t in transitions_sorted]
            )
        )])
        
        fig_sankey.update_layout(
            title="🔀 Top State Transitions Flow (Sankey Diagram)",
            height=500,
            font=dict(size=10)
        )
        
        fig_sankey.show()

---

## Section 3: Biometric-Demographic Correlation Analysis 🔗

**Cross-Dataset Pattern Discovery**

Identifying correlations between biometric and demographic updates

In [ ]:
# Analyze biometric-demographic correlation
correlation_data = cohort_analyzer.analyze_biometric_demographic_correlation(
    biometric_df, demographic_df
)

print("=" * 80)
print(" " * 20 + "🔗 BIOMETRIC-DEMOGRAPHIC CORRELATION ANALYSIS")
print("=" * 80)

if 'user_level' in correlation_data:
    ul = correlation_data['user_level']
    print(f"\n📊 USER-LEVEL ANALYSIS:")
    print("-" * 80)
    print(f"  Users with Both Updates:  {ul['users_with_both']:>25,}")
    print(f"  Users with Only Bio:      {ul['users_with_only_biometric']:>25,}")
    print(f"  Users with Only Demo:     {ul['users_with_only_demographic']:>25,}")
    print(f"  Overlap Percentage:       {ul['overlap_percentage']:>24.1f}%")
    print(f"  Avg Days Between Updates: {ul['avg_days_between_updates']:>25.1f}")

if 'state_level' in correlation_data:
    sl = correlation_data['state_level']
    print(f"\n🗺️ STATE-LEVEL CORRELATIONS:")
    print("-" * 80)
    
    # Show top 10 states by correlation
    correlations = [(state, data['correlation']) for state, data in sl.items() 
                    if isinstance(data, dict) and 'correlation' in data]
    correlations_sorted = sorted(correlations, key=lambda x: abs(x[1]), reverse=True)[:10]
    
    for state, corr in correlations_sorted:
        state_data = sl[state]
        print(f"\n  {state}:")
        print(f"    Correlation:            {corr:>25.3f}")
        print(f"    Bio Count:              {state_data['biometric_count']:>25,}")
        print(f"    Demo Count:             {state_data['demographic_count']:>25,}")

print("=" * 80)

In [ ]:
# Visualize correlation analysis
if 'user_level' in correlation_data and 'state_level' in correlation_data:
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('User Overlap Distribution', 'State-Level Correlations (Top 10)'),
        specs=[[{"type": "pie"}, {"type": "bar"}]]
    )
    
    # User overlap pie chart
    ul = correlation_data['user_level']
    labels = ['Both Updates', 'Only Biometric', 'Only Demographic']
    values = [ul['users_with_both'], ul['users_with_only_biometric'], ul['users_with_only_demographic']]
    colors = ['#66C2A5', '#FC8D62', '#8DA0CB']
    
    fig.add_trace(
        go.Pie(labels=labels, values=values, marker=dict(colors=colors),
               textinfo='label+percent', hole=0.3),
        row=1, col=1
    )
    
    # State-level correlations bar chart
    sl = correlation_data['state_level']
    correlations = [(state, data['correlation']) for state, data in sl.items() 
                    if isinstance(data, dict) and 'correlation' in data]
    correlations_sorted = sorted(correlations, key=lambda x: abs(x[1]), reverse=True)[:10]
    
    states = [c[0] for c in correlations_sorted]
    corr_values = [c[1] for c in correlations_sorted]
    colors_bar = ['#E63946' if c < 0 else '#06AED5' for c in corr_values]
    
    fig.add_trace(
        go.Bar(x=states, y=corr_values, marker=dict(color=colors_bar),
               text=[f"{c:.3f}" for c in corr_values], textposition='auto'),
        row=1, col=2
    )
    
    fig.update_layout(
        height=500,
        title_text="🔗 Biometric-Demographic Correlation Analysis",
        showlegend=False
    )
    
    fig.update_xaxes(title_text="", row=1, col=2, tickangle=-45)
    fig.update_yaxes(title_text="Correlation", row=1, col=2)
    
    fig.show()
    
    # Additional visualization: Time gap distribution
    if ul['avg_days_between_updates'] > 0:
        print(f"\n⏱️ Average time gap between biometric and demographic updates:")
        print(f"   {ul['avg_days_between_updates']:.1f} days ({ul['avg_days_between_updates']/30:.1f} months)")

---

## Section 4: Update Journey Flow Analysis 🛤️

**Temporal Progression & Geographic Movement**

Tracking how users progress through their update journey over time and space

In [ ]:
# Analyze update journey
journey_data = cohort_analyzer.analyze_update_journey(enrolment_df)

print("=" * 80)
print(" " * 25 + "🛤️ UPDATE JOURNEY ANALYSIS")
print("=" * 80)

print(f"\n📊 JOURNEY STATISTICS:")
print("-" * 80)
print(f"  Total Users Analyzed:     {journey_data['total_users']:>25,}")
print(f"  Total Update Events:      {journey_data['total_updates']:>25,}")
print(f"  Avg Updates per User:     {journey_data['avg_updates_per_user']:>25.2f}")
print(f"  Avg Journey Duration:     {journey_data['avg_journey_duration_days']:>25.1f} days")

if 'update_type_transitions' in journey_data:
    print(f"\n🔄 UPDATE TYPE TRANSITIONS:")
    print("-" * 80)
    transitions = journey_data['update_type_transitions']
    
    # Show top 10 transitions
    sorted_transitions = sorted(transitions.items(), key=lambda x: x[1], reverse=True)[:10]
    for (from_type, to_type), count in sorted_transitions:
        pct = (count / journey_data['total_updates']) * 100
        print(f"  {from_type} → {to_type:.<35} {count:>8,} ({pct:>5.1f}%)")

if 'geographic_movement' in journey_data:
    print(f"\n🗺️ GEOGRAPHIC MOVEMENT:")
    print("-" * 80)
    gm = journey_data['geographic_movement']
    print(f"  Users Who Moved States:   {gm['users_who_moved_states']:>25,}")
    print(f"  Users Who Moved Districts:{gm['users_who_moved_districts']:>25,}")
    print(f"  State Movement Rate:      {gm['state_movement_rate']:>24.1f}%")
    print(f"  District Movement Rate:   {gm['district_movement_rate']:>24.1f}%")

if 'temporal_patterns' in journey_data:
    print(f"\n📅 TEMPORAL PATTERNS:")
    print("-" * 80)
    tp = journey_data['temporal_patterns']
    if 'peak_update_month' in tp:
        print(f"  Peak Update Month:        {tp['peak_update_month']:>25}")
    if 'peak_update_day_of_week' in tp:
        print(f"  Peak Update Day:          {tp['peak_update_day_of_week']:>25}")

print("=" * 80)

In [ ]:
# Visualize update journey with Sankey diagram
if 'update_type_transitions' in journey_data:
    transitions = journey_data['update_type_transitions']
    
    # Get top 20 transitions for cleaner visualization
    sorted_transitions = sorted(transitions.items(), key=lambda x: x[1], reverse=True)[:20]
    
    # Extract unique update types
    all_types = set()
    for (from_type, to_type), _ in sorted_transitions:
        all_types.add(from_type)
        all_types.add(to_type)
    
    node_list = list(all_types)
    node_dict = {node: idx for idx, node in enumerate(node_list)}
    
    # Prepare Sankey data
    source = [node_dict[from_type] for (from_type, to_type), _ in sorted_transitions]
    target = [node_dict[to_type] for (from_type, to_type), _ in sorted_transitions]
    value = [count for _, count in sorted_transitions]
    
    # Create color palette
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', 
              '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] * 5
    
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_list,
            color=colors[:len(node_list)]
        ),
        link=dict(
            source=source,
            target=target,
            value=value,
            label=[f"{v:,}" for v in value]
        )
    )])
    
    fig.update_layout(
        title="🛤️ Update Type Journey Flow (Top 20 Transitions)",
        height=600,
        font=dict(size=11)
    )
    
    fig.show()

# Visualize geographic movement
if 'geographic_movement' in journey_data:
    gm = journey_data['geographic_movement']
    
    labels = ['Moved States', 'Moved Districts Only', 'No Movement']
    values = [
        gm['users_who_moved_states'],
        gm['users_who_moved_districts'] - gm['users_who_moved_states'],
        journey_data['total_users'] - gm['users_who_moved_districts']
    ]
    
    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        marker=dict(colors=['#E63946', '#F4A261', '#2A9D8F']),
        textinfo='label+percent+value',
        textfont=dict(size=12)
    )])
    
    fig.update_layout(
        title="🗺️ Geographic Movement Distribution",
        height=500
    )
    
    fig.show()

---

## Section 5: Multi-Dimensional Segmentation 📊

**Advanced User Segmentation**

Creating actionable user segments based on multiple dimensions

In [ ]:
# Initialize Segmentation Engine
segmentation_engine = SegmentationEngine()

# Perform multi-dimensional segmentation
segments = segmentation_engine.multi_dimensional_segmentation(demographic_df)

print("=" * 80)
print(" " * 20 + "📊 MULTI-DIMENSIONAL SEGMENTATION ANALYSIS")
print("=" * 80)

for dimension, segment_data in segments.items():
    print(f"\n{'='*80}")
    print(f"  DIMENSION: {dimension.upper().replace('_', ' ')}")
    print(f"{'='*80}")
    
    for segment_name, stats in segment_data.items():
        print(f"\n  📍 {segment_name}:")
        print(f"    Count:                  {stats['count']:>25,}")
        print(f"    Percentage:             {stats['percentage']:>24.1f}%")
        
        if 'avg_age' in stats:
            print(f"    Average Age:            {stats['avg_age']:>25.1f} years")
        
        if 'top_states' in stats and stats['top_states']:
            print(f"    Top States:")
            for state, count in list(stats['top_states'].items())[:3]:
                print(f"      {state:.<42} {count:>10,}")
        
        if 'top_update_types' in stats and stats['top_update_types']:
            print(f"    Top Update Types:")
            for update_type, count in list(stats['top_update_types'].items())[:3]:
                print(f"      {update_type:.<42} {count:>10,}")

print("\n" + "=" * 80)

In [ ]:
# Visualize multi-dimensional segmentation
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=list(segments.keys()),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

dimension_list = list(segments.keys())
colors_palette = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#F7DC6F']

for idx, dimension in enumerate(dimension_list[:4]):  # Max 4 dimensions for 2x2 grid
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    
    segment_data = segments[dimension]
    segment_names = list(segment_data.keys())
    segment_counts = [segment_data[s]['count'] for s in segment_names]
    segment_percentages = [segment_data[s]['percentage'] for s in segment_names]
    
    fig.add_trace(
        go.Bar(
            x=segment_names,
            y=segment_counts,
            text=[f"{p:.1f}%" for p in segment_percentages],
            textposition='auto',
            marker_color=colors_palette[idx],
            showlegend=False
        ),
        row=row, col=col
    )
    
    fig.update_xaxes(title_text="Segment", row=row, col=col, tickangle=-45)
    fig.update_yaxes(title_text="Count", row=row, col=col)

fig.update_layout(
    height=800,
    title_text="📊 Multi-Dimensional Segmentation Overview",
    showlegend=False
)

fig.show()

# Create RFM-like segmentation for deeper analysis
print("\n" + "=" * 80)
print(" " * 25 + "🎯 RFM-LIKE SEGMENTATION")
print("=" * 80)

rfm_segments = segmentation_engine.create_rfm_segments(enrolment_df)

for segment_name, user_ids in rfm_segments.items():
    pct = (len(user_ids) / len(enrolment_df['user_id'].unique())) * 100
    print(f"\n  {segment_name:.<50} {len(user_ids):>10,} ({pct:>5.1f}%)")

print("\n" + "=" * 80)

---

## 📋 Executive Summary: Cohort & Segmentation Insights

### Key Findings

**1. Children vs Adults Behavioral Differences**
- Distinct update patterns and geographic coverage between age groups
- Strategic implications for targeted outreach and service delivery

**2. State Transition Patterns**
- Clear lifecycle progression through update states
- Predictable transition probabilities enable proactive service planning

**3. Cross-Dataset Correlations**
- Strong relationship between biometric and demographic updates
- User overlap insights guide integrated service approaches

**4. Update Journey Dynamics**
- Temporal and geographic movement patterns identified
- Journey flow analysis reveals optimization opportunities

**5. Multi-Dimensional Segmentation**
- Age, geography, time, and update type segments created
- RFM-like scoring enables personalized engagement strategies

### Strategic Recommendations

1. **Age-Specific Campaigns**: Design targeted interventions for children vs adults based on behavioral differences
2. **Predictive Resource Allocation**: Use transition matrices to forecast demand and optimize resource deployment
3. **Integrated Service Delivery**: Leverage biometric-demographic correlation to streamline update processes
4. **Journey Optimization**: Reduce friction in update paths by addressing identified bottlenecks
5. **Personalized Engagement**: Apply multi-dimensional segmentation for customized communication and services

### Impact Potential

- **Improved User Experience**: Targeted services based on cohort characteristics
- **Operational Efficiency**: Predictive insights enable proactive planning
- **Higher Engagement**: Personalized approaches increase update completion rates
- **Data-Driven Decisions**: Segmentation framework supports evidence-based policy making